In [1]:
import theano
from theano import tensor as T
import lasagne
from lasagne.layers import *
from lasagne.objectives import *
from lasagne.nonlinearities import *
import cPickle as pickle
import gzip
import numpy as np

In [2]:
def get_net():
    l_in = InputLayer((None, 784))
    l_dense = DenseLayer(l_in, num_units=256)
    l_dense2 = DenseLayer(l_dense, num_units=128)
    l_softmax = DenseLayer(l_dense2, num_units=10, nonlinearity=softmax)
    return l_softmax

Load in MNIST data.

In [3]:
with gzip.open("./mnist.pkl.gz") as f:
    train_data, valid_data, _ = pickle.load(f)

In [4]:
X_train, y_train = train_data
X_valid, y_valid = valid_data

Define the worker, which will be run in its own thread.

In [5]:
def iterator(X,y,bs):
    b = 0
    while True:
        if b*bs >= X.shape[0]:
            break
        yield X[(b*bs):(b+1)*bs], y[(b*bs):(b+1)*bs]
        b += 1

In [11]:
def worker(X_train, y_train, net_fn, num_epochs, queue):
    """
    X_train: the chunk of training data this worker is meant to operate on
    """
    l_out = net_fn()
    X = T.fmatrix('X')
    y = T.ivector('y')
    net_out = get_output(l_out, X)
    loss = categorical_crossentropy(net_out, y).mean()
    params = get_all_params(l_out, trainable=True)
    grads = T.grad(loss, params)
    grads_fn = theano.function([X,y], grads)
    print "num epochs", num_epochs
    for epoch in range(num_epochs):
        print "epoch:", epoch
        for X_batch, y_batch in iterator(X_train, y_train, bs=32):
            # this breaks the epoch for loop
            this_grads = grads_fn(X_batch, y_batch)
            queue.put(this_grads)

In [161]:
#worker(X_train[0:100].astype("float32"), y_train[0:100].astype("int32"), get_net, 100, None)

In [7]:
import multiprocessing
queue = multiprocessing.Queue(maxsize=100)

In [19]:
p = multiprocessing.Process(target=worker, 
        args=(X_train[0:100].astype("float32"), y_train[0:100].astype("int32"), get_net, 100, queue,))
p.start()

num epochs 100
epoch: 0


In [20]:
p.join()

KeyboardInterrupt: 

In [18]:
gtmp = queue.get()
gtmp

[array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 array([ -3.02678797e-04,   5.58130128e-03,  -1.54949869e-04,
         -3.35858863e-03,   1.15703361e-02,  -1.51021615e-02,
         -5.44662625e-03,   7.08543004e-03,  -2.43729588e-03,
          1.58452232e-03,  -3.08737592e-03,  -6.63151591e-03,
         -4.71173387e-03,  -9.86536378e-03,  -1.07615016e-02,
         -4.76057029e-03,   1.01019259e-03,   3.81981194e-03,
          5.93102281e-03,   1.90384393e-03,   2.35445730e-02,
         -4.46881701e-03,  -1.55612358e-02,   3.67632513e-03,
          8.75471367e-03,   6.77829442e-03,   2.36708644e-02,
         -2.87656427e-03,  -1.63856087e-03,   1.20956689e-03,
          3.92618672e-03,  -3.81946242e-03,  -8.68809402e-03,
         -1.58036615e-03,  -4.63

In [104]:
?p.terminate()

----

In [93]:
master_current_params = None

In [99]:
def master_worker(X_valid, y_valid, net_fn):
    """
    X_valid:
    y_valid:
    """
    global master_current_params
    l_out = net_fn()
    params = get_all_params(l_out, trainable=True)
    X = T.fmatrix('X')
    y = T.ivector('y')
    net_out = get_output(l_out, X)
    out_fn = theano.function([X], net_out)
    for iter in range(100):
        # ok, let's try and get a grad object from the queue
        # and then update our params before evaluating on
        # the validation set
        grads = queue.get()
        print "got grads, doing an update..."
        for i in range(len(params)):
            # do sgd on this param
            params[i].set_value( params[i].get_value() - 0.01*grads[i])
        
        preds = []
        for X_batch, y_batch in iterator(X_valid, y_valid, bs=32):
            this_preds = np.argmax(out_fn(X_batch),axis=1)
            preds += this_preds.tolist()
        preds = np.asarray(preds)
        valid_acc = (preds == y_valid).mean()
        print valid_acc
        master_current_params = get_all_param_values(l_out)
    

In [100]:
master_worker(X_valid[0:100].astype("float32"), y_valid[0:100].astype("float32"), get_net)

KeyboardInterrupt: 

In [84]:
len(master_current_params)

6

In [88]:
arr = [1,2,3,4,5]

In [89]:
arr.pop()

5

In [90]:
for t in arr.pop():
    print t

TypeError: 'int' object is not iterable

In [167]:
%%bash
du -h /storeSSD/cbeckham/deep_rl

du: /storeSSD/cbeckham/deep_rl: No such file or directory
